# Additional Synchronised Traces - Orthopaedic Ward - Hospital Efficiency Project

This is the orthopaedic surgery model developed as part of the hospital efficiency project.

original model author = Harper, Alison and Monks, Thomas

license = MIT

title = Hospital Efficiency Project  Orthopaedic Planning Model Discrete-Event Simulation

url = https://github.com/AliHarp/HEP

It has been used as a test case here to allow the development and testing of several key features of the event log animations:

- adding of logging to a model from scratch

- ensuring the requirement to use simpy stores instead of simpy resources doesn't prevent the uses of certain common modelling patterns (in this case, conditional logic where patients will leave the system if a bed is not available within a specified period of time)

- displaying different icons for different classes of patients

- displaying custom resource icons

- displaying additional static information as part of the icon (in this case, whether the client's discharge is delayed)

- displaying information that updates with each animation step as part of the icon (in this case, the LoS of the patient at each time point)

In [1]:
import gc
import time
import datetime as dt
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from examples.example_13_additional_synchronised_traces_method_1.simulation_execution_functions import multiple_replications
from examples.example_13_additional_synchronised_traces_method_1.model_classes import Scenario, Schedule
from vidigi.prep import reshape_for_animations, generate_animation_df
from vidigi.animation import generate_animation
from plotly.subplots import make_subplots


TRACE = True
debug_mode=True

schedule = Schedule()



4 theatres

5 day/week

Each theatre has three sessions per day:

Morning: 1 revision OR 2 primary

Afternoon: 1 revision OR 2 primary

Evening: 1 primary

40 ring-fenced beds for recovery from these operations


In [2]:
(pd.DataFrame.from_dict(schedule.sessions_per_weekday, orient="index")
        .rename(columns={0: "Sessions"}).merge(

        pd.DataFrame.from_dict(schedule.theatres_per_weekday, orient="index")
            .rename(columns={0: "Theatre Capacity"}),
            left_index=True, right_index=True

        ).merge(

        pd.DataFrame.from_dict(schedule.allocation, orient="index"),
        left_index=True, right_index=True

        ))

,Sessions,Theatre Capacity,0,1,2
Monday,3,4,2P_or_1R,2P_or_1R,1P
Tuesday,3,4,2P_or_1R,2P_or_1R,1P
Wednesday,3,4,2P_or_1R,2P_or_1R,1P
Thursday,3,4,2P_or_1R,2P_or_1R,1P
Friday,3,4,2P_or_1R,2P_or_1R,1P
Saturday,0,0,None,None,None
Sunday,0,0,None,None,None


In [3]:
n_beds = 40

primary_hip_los = 4.4

primary_knee_los = 4.7

revision_hip_los = 6.9

revision_knee_los = 7.2

unicompart_knee_los = 2.9

los_delay = 16.5
los_delay_sd = 15.2

prop_delay = 0.076

replications = 30
runtime = 60
warmup=7

args = Scenario(schedule=schedule,
                primary_hip_mean_los=primary_hip_los,
                primary_knee_mean_los=primary_knee_los,
                revision_hip_mean_los=revision_hip_los,
                revision_knee_mean_los=revision_knee_los,
                unicompart_knee_mean_los=unicompart_knee_los,
                prob_ward_delay=prop_delay,
                n_beds=n_beds,
                delay_post_los_mean=los_delay,
                delay_post_los_sd=los_delay_sd
                )


results = multiple_replications(
                return_detailed_logs=True,
                scenario=args,
                n_reps=replications,
                results_collection=runtime
            )




# Join the event log with a list of patients to add a column that will determine
# the icon set used for a patient (in this case, we want to distinguish between the
# knee/hip patients)
event_log = results[4]
event_log = event_log[event_log['rep'] == 1]
event_log['patient'] = event_log['patient'].astype('str') + event_log['pathway']

primary_patients = results[2]
primary_patients = primary_patients[primary_patients['rep'] == 1]
primary_patients['patient class'] = primary_patients['patient class'].str.title()
primary_patients['ID'] = primary_patients['ID'].astype('str') + primary_patients['patient class']

revision_patients = results[3]
revision_patients = revision_patients[revision_patients['rep'] == 1]
revision_patients['patient class'] = revision_patients['patient class'].str.title()
revision_patients['ID'] = revision_patients['ID'].astype('str') + revision_patients['patient class']

full_log_with_patient_details = event_log.merge(pd.concat([primary_patients, revision_patients]),
                                                    how="left",
                                                left_on=["patient", "pathway"],
                                                right_on=["ID", "patient class"]).reset_index(drop=True).drop(columns="ID")

pid_table = full_log_with_patient_details[['patient']].drop_duplicates().reset_index(drop=True).reset_index(drop=False).rename(columns={'index': 'pid'})

full_log_with_patient_details = full_log_with_patient_details.merge(pid_table, how='left', on='patient').drop(columns='patient').rename(columns={'pid':'patient'})


C:\Users\Sammi\AppData\Local\Temp\ipykernel_25884\3505082114.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_log['patient'] = event_log['patient'].astype('str') + event_log['pathway']
C:\Users\Sammi\AppData\Local\Temp\ipykernel_25884\3505082114.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_patients['patient class'] = primary_patients['patient class'].str.title()
C:\Users\Sammi\AppData\Local\Temp\ipykernel_25884\3505082114.py:55: SettingWithCopyWarning: 
A value is trying to be set 

In [4]:

event_position_df = pd.DataFrame([
            # {'event': 'arrival', 'x':  10, 'y': 250, 'label': "Arrival" },

            # Triage - minor and trauma
            {'event': 'enter_queue_for_bed',
                'x':  200, 'y': 650, 'label': "Waiting for<br>Availability of<br>Bed to be Confirmed<br>Before Surgery" },

            {'event': 'no_bed_available',
                'x':  600, 'y': 650, 'label': "No Bed<br>Available:<br>Surgery Cancelled" },

            {'event': 'post_surgery_stay_begins',
                'x':  650, 'y': 220, 'resource':'n_beds', 'label': "In Bed:<br>Recovering from<br>Surgery" },

            {'event': 'discharged_after_stay',
                'x':  670, 'y': 50, 'label': "Discharged from Hospital<br>After Recovery"}
            # {'event': 'exit',
            #  'x':  670, 'y': 100, 'label': "Exit"}

            ])


In [5]:
full_patient_df = reshape_for_animations(full_log_with_patient_details,
                                         entity_col_name="patient",
                                            every_x_time_units=1,
                                            limit_duration=runtime,
                                            step_snapshot_max=50,
                                            debug_mode=debug_mode
                                            )

if debug_mode:
    print(f'Reshaped animation dataframe finished construction at {time.strftime("%H:%M:%S", time.localtime())}')


Iteration through time-unit-by-time-unit logs complete 17:02:53
Snapshot df concatenation complete at 17:02:53
Reshaped animation dataframe finished construction at 17:02:53


In [6]:
full_patient_df_plus_pos = generate_animation_df(
                            full_entity_df=full_patient_df,
                            entity_col_name="patient",
                            event_position_df=event_position_df,
                            wrap_queues_at=20,
                            wrap_resources_at=40,
                            step_snapshot_max=50,
                            gap_between_entities=15,
                            gap_between_resources=15,
                            gap_between_queue_rows=175,
                            gap_between_resource_rows=175,
                            debug_mode=debug_mode
                    )


Placement dataframe finished construction at 17:02:53


In [7]:
def set_icon(row):
    if row["surgery type"] == "p_knee":
        return "🦵<br>1️⃣<br> "
    elif row["surgery type"] == "r_knee":
        return "🦵<br>♻️<br> "
    elif row["surgery type"] == "p_hip":
        return "🕺<br>1️⃣<br> "
    elif row["surgery type"] == "r_hip":
        return "🕺<br>♻️<br> "
    elif row["surgery type"] == "uni_knee":
        return "🦵<br>✳️<br> "
    else:
        return f"CHECK<br>{row['icon']}"

full_patient_df_plus_pos = full_patient_df_plus_pos.assign(icon=full_patient_df_plus_pos.apply(set_icon, axis=1))

# TODO: Check why this doesn't seem to be working quite right for the 'discharged after stay'
# step. e.g. 194Primary is discharged on 28th July showing a LOS of 1 but prior to this shows a LOS of 9.
def add_los_to_icon(row):
    if row["event"] == "post_surgery_stay_begins":
        return f'{row["icon"]}<br>{row["snapshot_time"]-row["time"]:.0f}'
    elif row["event"] == "discharged_after_stay":
        return f'{row["icon"]}<br>{row["los"]:.0f}'
    else:
        return row["icon"]

full_patient_df_plus_pos = full_patient_df_plus_pos.assign(icon=full_patient_df_plus_pos.apply(add_los_to_icon, axis=1))


def indicate_delay_via_icon(row):
    if row["delayed discharge"] is True:
        return f'{row["icon"]}<br>*'
    else:
        return f'{row["icon"]}<br> '

full_patient_df_plus_pos = full_patient_df_plus_pos.assign(icon=full_patient_df_plus_pos.apply(indicate_delay_via_icon, axis=1))

cancelled_due_to_no_bed_available = len(full_log_with_patient_details[full_log_with_patient_details['event'] == "no_bed_available"]["patient"].unique())
total_patients = len(full_log_with_patient_details["patient"].unique())

cancelled_perc = cancelled_due_to_no_bed_available/total_patients

# st.markdown(f"Surgeries cancelled due to no bed being available in time: {cancelled_perc:.2%} ({cancelled_due_to_no_bed_available} of {total_patients})")

# st.markdown(
#     """
#     **Key**:

#     🦵1️⃣: Primary Knee

#     🦵♻️: Revision Knee

#     🕺1️⃣: Primary Hip

#     🕺♻️: Revision Hip

#     🦵✳️: Primary Unicompartment Knee

#     An asterisk (*) indicates that the patient has a delayed discharge from the ward.

#     The numbers below patients indicate their length of stay.

#     Note that the "No Bed Available: Surgery Cancelled" and "Discharged from Hospital after Recovery" stages in the animation are lagged by one day.
#     For example, on the 2nd of July, this will show the patients who had their surgery cancelled on 1st July or were discharged on 1st July.
#     These steps are included to make it easier to understand the destinations of different clients, but due to the size of the simulation step shown (1 day) it is difficult to demonstrate this differently.
#     """
# )


In [8]:
fig = generate_animation(
        full_entity_df_plus_pos=full_patient_df_plus_pos,
        entity_col_name="patient",
        event_position_df=event_position_df,
        scenario=args,
        plotly_height=950,
        plotly_width=1000,
        override_x_max=800,
        override_y_max=1000,
        entity_icon_size=14,
        text_size=14,
        wrap_resources_at=40,
        gap_between_resources=15,
        include_play_button=True,
        add_background_image=None,
        # we want the stage labels, but due to a bug
        # when we add in additional animated traces later,
        # they will disappear - so better to leave them out here
        # and then re-add them manually
        display_stage_labels=False,
        custom_resource_icon="🛏️",
        time_display_units="d",
        simulation_time_unit="days",
        start_date="2022-06-27",
        setup_mode=False,
        frame_duration=1500, #milliseconds
        frame_transition_duration=1000, #milliseconds
        debug_mode=False
    )

fig

In [ ]:
counts_not_avail = full_patient_df_plus_pos[full_patient_df_plus_pos['event']=='no_bed_available'][['snapshot_time','patient']].groupby('snapshot_time').agg('count')
counts_not_avail = counts_not_avail.reset_index().merge(full_patient_df_plus_pos[['snapshot_time']].drop_duplicates(), how='right').sort_values('snapshot_time')
counts_not_avail['patient'] = counts_not_avail['patient'].fillna(0)
counts_not_avail['running_total'] = counts_not_avail['patient'].cumsum()

counts_ops_completed = full_patient_df_plus_pos[full_patient_df_plus_pos['event']=='post_surgery_stay_begins'][['snapshot_time','patient']].drop_duplicates('patient').groupby('snapshot_time').agg('count')
counts_ops_completed = counts_ops_completed.reset_index().merge(full_patient_df_plus_pos[['snapshot_time']].drop_duplicates(), how='right').sort_values('snapshot_time')
counts_ops_completed['patient'] = counts_ops_completed['patient'].fillna(0)
counts_ops_completed['running_total'] = counts_ops_completed['patient'].cumsum()

counts_not_avail = counts_not_avail.merge(counts_ops_completed.rename(columns={'running_total':'completed'}), how="left", on="snapshot_time")
counts_not_avail['perc_slots_lost'] = counts_not_avail['running_total'] / (counts_not_avail['running_total'] + counts_not_avail['completed'])

#####################################################
# Adding additional animation traces
#####################################################

## First, add each trace so it will show up initialls

# Due to issues detailed in the following SO threads, it's essential to initialize the traces
# outside of the frames argument else they will not show up at all (or show up intermittently)
# https://stackoverflow.com/questions/69867334/multiple-traces-per-animation-frame-in-plotly
# https://stackoverflow.com/questions/69367344/plotly-animating-a-variable-number-of-traces-in-each-frame-in-r
# TODO: More explanation and investigation needed of why sometimes traces do and don't show up after being added in
# via this method. Behaviour seems very inconsistent and not always logical (e.g. order you put traces in to the later
# loop sometimes seems to make a difference but sometimes doesn't; making initial trace transparent sometimes seems to
# stop it showing up when added in the frames but not always; sometimes the initial trace doesn't disappear).

# ==============================================================================
# 1. ADD ALL TRACES (STATIC AND DYNAMIC) TO THE FIGURE
# We will add them in order: first all dynamic traces, then all static ones.
# This makes indexing easier to manage.
# ==============================================================================

# Let's assume your original figure 'fig' has 2 dynamic traces already.
# These will be at index 0 and 1.

# --- Add Placeholders for NEW DYNAMIC Traces ---

# DYNAMIC TRACE: Operations Completed (will be at index 2)
# We add its initial state. It's invisible (opacity=0) until the animation starts.
fig.add_trace(go.Scatter(
    x=[100], y=[30],
    text=f"Operations Completed: {int(counts_ops_completed['running_total'][0])}",
    mode='text', textfont=dict(size=20),
    opacity=0, # Start invisible
    showlegend=False,
))

# DYNAMIC TRACE: Slots Lost (will be at index 3)
fig.add_trace(go.Scatter(
    x=[600], y=[850],
    text="",
    mode='text', textfont=dict(size=20),
    showlegend=False,
))

# DYNAMIC TRACE: Animated Subplot Line (will be at index 4)
# Add an empty trace as a placeholder. It will be populated by the animation frames.
fig.add_trace(go.Scatter(
    x=[], y=[],
    mode="lines", showlegend=False,
    name="line_subplot",
    xaxis='x2', yaxis='y2'
))

# --- Add STATIC Traces ---
# These are added once and will not be included in the animation frames.

# STATIC TRACE: Bed Icons (will be at index 5)
# Note: Replaced the confusing fig.add_trace(fig.data[1]) with an explicit definition.
# You should replace the x and y with the actual coordinates of your bed icons.
fig.add_trace(go.Scatter(
    x=[...], # <-- Add your bed icon X coordinates here
    y=[...], # <-- Add your bed icon Y coordinates here
    mode='markers', # Or whatever mode you used for the beds
    showlegend=False,
    # ... other styling for bed icons
))

# STATIC TRACE: Event Labels (will be at index 6)
# This is now added only once and will persist through the animation.
fig.add_trace(go.Scatter(
    x=[pos+10 for pos in event_position_df['x'].to_list()],
    y=event_position_df['y'].to_list(),
    mode="text", name="",
    text=event_position_df['label'].to_list(),
    textposition="middle right",
    hoverinfo='none',
    showlegend=False,
))

# STATIC TRACE: Subplot Background Line (will be at index 7)
fig.add_trace(go.Scatter(
    x=counts_not_avail['snapshot_time'],
    y=counts_not_avail['patient_x'],
    mode='lines', showlegend=False,
    line=dict(color='lightgrey'), # Use a color to distinguish it
    opacity=0.4,
    xaxis="x2", yaxis="y2"
), row=2, col=1)

# Ensure text sizes are correct
fig.update_traces(textfont_size=14)

# (Your subplot setup code for layout and axes remains the same...)
# Now set up the desired subplot layout
sp = make_subplots(rows=2, cols=1, row_heights=[0.85, 0.15], subplot_titles=("", "Daily lost slots"))
fig.layout['xaxis']['domain'] = sp.layout['xaxis']['domain']
fig.layout['yaxis']['domain'] = sp.layout['yaxis']['domain']
fig.layout['xaxis2'] = sp.layout['xaxis2']
fig.layout['yaxis2'] = sp.layout['yaxis2']
fig._grid_ref = sp._grid_ref


# ==============================================================================
# 2. BUILD THE ANIMATION FRAMES
# The frames will now ONLY contain data for the DYNAMIC traces.
# ==============================================================================

for i, frame in enumerate(fig.frames):
    # This is the data for the original 2 animated traces
    original_data = frame.data

    # This tuple contains the data for the NEW dynamic traces we added,
    # in the same order we added them (Ops Completed, Slots Lost, Subplot Line).
    new_dynamic_data = (
        # Data for "Operations Completed" text (updates trace 2)
        go.Scatter(
            text=f"Operations Completed: {int(counts_ops_completed['running_total'][i])}",
            opacity=1 # Make it visible
        ),
        # Data for "Slots Lost" text (updates trace 3)
        go.Scatter(
            text=f"Total slots lost: {int(counts_not_avail['running_total'][i])}<br>({counts_not_avail['perc_slots_lost'][i]:.1%})"
        ),
        # Data for animated subplot line (updates trace 4)
        go.Scatter(
            x=counts_not_avail['snapshot_time'][0: i+1].values,
            y=counts_not_avail['patient_x'][0: i+1].values,
        ),
    )

    # Combine the original animation data with the new dynamic data
    frame.data = original_data + new_dynamic_data

# ==============================================================================
# 3. SET THE TRACES TO ANIMATE
# This list now ONLY contains the indices of our dynamic traces.
# ==============================================================================

# Original traces (0, 1) + New dynamic traces (2, 3, 4)
dynamic_trace_indices = [0, 1, 2, 3, 4]

for frame in fig.frames:
    frame.traces = dynamic_trace_indices

# The static traces (5, 6, 7) are NOT listed here, so the animation
# will leave them untouched.

fig